<a href="https://colab.research.google.com/github/ykitaguchi77/GenerativeAgents/blob/main/Chapter3_promptengineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chapter3 Prompt engineering**



In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [3]:
!pip install openai==1.40.6 httpx==0.27.2 --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00


## プロンプトエンジニアリングとは

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "プロンプトエンジニアリングとは"},
    ],
)
print(response.choices[0].message.content)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "質問に100文字程度で答えてください。"},
        {"role": "user", "content": "プロンプトエンジニアリングとは"},
    ],
)
print(response.choices[0].message.content)

プロンプトエンジニアリングは、AIに特定の結果を生成させるために最適な質問や命令を設計する技術です。効果的なプロンプトを作成することで、AIの応答の質や関連性を向上させることができます。主に自然言語処理の分野で活用されています。


## 3.3. プロンプトの構成要素の基本


In [ ]:
prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt.format(dish=dish)},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

カレーのレシピを考えました。ここでは、基本的なチキンカレーのレシピを紹介します。お好みに応じて他の具材や香辛料を追加しても美味しく作れます。

### 材料（4人分）

- 鶏もも肉：400g
- 玉ねぎ：2個
- にんにく：2片
- しょうが：1片
- トマト：2個（またはカットトマト缶400g）
- カレー粉：大さじ2
- サラダ油：大さじ2
- 塩：適量
- 胡椒：適量
- 水：500ml
- ココナッツミルク（お好みで）：200ml
- ガラムマサラ（お好みで）：小さじ1
- ピーマンやニンジンなどの野菜（お好みで）：適量
- ご飯（米）：適量

### 作り方

1. **材料の下ごしらえ**：
   - 鶏もも肉は一口大に切ります。
   - 玉ねぎ、にんにく、しょうがはみじん切りにします。
   - トマトはざく切りにします。
   - お好みの野菜も適宜切っておきます。

2. **鶏肉の下処理**：
   - 鍋にサラダ油を熱し、鶏肉を加えて表面が白くなるまで炒めます。塩と胡椒を少々振りかけて下味をつけます。

3. **香味野菜の調理**：
   - 鶏肉が白くなったら、みじん切りにした玉ねぎを加え、透明になるまで炒めます。
   - 続いて、にんにくとしょうがを加え、香りが立つまで炒めます。

4. **スパイスとトマトの追加**：
   - カレー粉を加えて全体を混ぜ、中火で1分ほど炒めます。
   - トマトを加え、柔らかくなるまで煮ます。

5. **煮込み**：
   - 水を加え、沸騰したらアクを取り除き、中火にして約20分煮込みます。
   - （お好みで）ピーマンやニンジンなどの野菜を加え、さらに10分程度煮込みます。

6. **仕上げ**：
   - ココナッツミルクを加え、全体を混ぜて5分ほど煮込みます。
   - 味見をして、塩や胡椒で調整します。
   - 最後にガラムマサラを加えて、火を止めます。

7. **盛り付け**：
   - ご飯を皿に盛り、カレーをかけていただきます。

### おすすめのトッピング

- お好みでパセリやレモンを添えても美味しいです。
- ナンやライスと一緒に楽しむとさらに満足感が増します。

ぜひ、おいしいカレーを作ってみてください！


In [ ]:
prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''
prompt.format(dish="カレー")

'以下の料理のレシピを考えてください。\n\n料理名: """\nカレー\n"""\n'

In [ ]:
prompt = """\
ユーザーが入力した料理のレシピを考えてください。

料理名: '''
{dish}
'''
"""


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "ユーザーが入力した料理のレシピを考えてください。"},
            {"role": "user", "content": f"{dish}"},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

## 出力形式を指定する


In [ ]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
)
print(response.choices[0].message.content)

{
  "材料": ["鶏肉", "玉ねぎ", "にんじん", "じゃがいも", "カレールー", "水", "塩", "胡椒"],
  "手順": [
    "鶏肉、玉ねぎ、にんじん、じゃがいもを食べやすい大きさに切る。",
    "鍋に油を熱し、玉ねぎを炒め、透明になるまで炒め続ける。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、全体を混ぜて軽く炒める。",
    "水を加え、煮立ったらアクを取り、弱火で20分程度煮る。",
    "カレールーを加え、よく溶かして香りが立つまで煮込み、塩と胡椒で味を調える。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


#**プロンプトテクニック**

### Zero-shot プロンプティング


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "入力をポジティブ・ネガティブ・中立のどれかに分類してください。",
        },
        {
            "role": "user",
            "content": "ChatGPTはとても便利だ",
        },
    ],
)
print(response.choices[0].message.content)

ポジティブ


### Few-shot prompting

In [5]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

はい、ChatGPTは多くの情報を提供したり、質問に答えたりするのに非常に便利なツールです。様々なトピックについての会話をサポートしたり、アイデアを出したりすることができます。他に何か知りたいことがあれば教えてください。


In [6]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "AIの進化はすごい"},
        {"role": "assistant", "content": "true"},
        {"role": "user", "content": "今日は良い天気だ"},
        {"role": "assistant", "content": "false"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

true


### Zero-shot Chain-of-Thought プロンプティング


In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "回答だけ一言で出力してください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)

4


In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "ステップバイステップで考えてください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)

この数式をステップバイステップで解いていきましょう。

数式: \( 10 + 2 * 3 - 4 * 2 \)

1. まずは掛け算から計算します。
   - \( 2 * 3 = 6 \)
   - \( 4 * 2 = 8 \)

2. 数式を置き換えます。
   - \( 10 + 6 - 8 \)

3. 次に足し算を行います。
   - \( 10 + 6 = 16 \)

4. 最後に引き算を行います。
   - \( 16 - 8 = 8 \)

したがって、最終的な答えは \( 8 \) です。
